# Data Preparation, Pipelines & Model 

In [2]:
# Modules importeren
import pandas as pd
from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt
import seaborn as sns 
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.compose import ColumnTransformer

# Dataset importeren 
df = pd.read_csv("/Users/odessa/Desktop/Applied Data Science & AI/Data Science/Code Inleiding data science/song_data.csv")

# Target variabele maken 
target = 'song_popularity'
#df.drop(columns=["song_name"], inplace=True) # inplace=True veranderd de originele dataframe zonder nieuwe dataframe te maken 

### Phase 3: Data Preparation

In [3]:
# 2 nummers droppen
df = df.drop([7119, 11171]).reset_index(drop=True)

In [ ]:
print(f"Totaal aantal waardes in de dataframe vóór het verwijderen van dubbele waardes uit song_name en song_duration_ms: {len(df)}")

# Dubbele waardes droppen van song_name en song_duration 
# Als ik alleen song_name duplicates zou verwijderen, zou ik misschien covers van nummers verwijderen, dus daarom check ik ook de song_duration 
df.drop_duplicates(subset=['song_name', 'song_duration_ms'], inplace = True)
print(f"Totaal aantal waardes in de dataframe na verwijderen van dubbele waardes uit song_name en song_duration_ms: {len(df)}")

Totaal aantal waardes in de dataframe vóór het verwijderen van dubbele waardes uit song_name en song_duration_ms: 18833
Totaal aantal waardes in de dataframe na verwijderen van dubbele waardes uit song_name en song_duration_ms: 14466


In [ ]:
df.drop(columns=["song_name"], inplace=True) # inplace=True veranderd de originele dataframe zonder nieuwe dataframe te maken 

In [ ]:
# class maken en daarna in pipeline zetten 
def ikr_grenzen_berekenen(df: pd.DataFrame, kolommen=None):
    """
    Berekent voor elke numerieke kolom de onder- en bovengrens 
    op basis van interkwartielafstand (IKR).
    """
    if kolommen is None: 
        kolommen = df.select_dtypes(include=['int64', 'float64']).columns
    
    grenzen = {}

    for kolom in kolommen: 
        Q1 = df[kolom].quantile(0.25)
        Q3 = df[kolom].quantile(0.75)
        IKR = Q3 - Q1 
        ondergrens = Q1 - 1.5 * IKR
        bovengrens = Q3 + 1.5 * IKR
        grenzen[kolom] = (ondergrens, bovengrens)

    return grenzen 


def uitschieters_detecteren(df: pd.DataFrame, grenzen: dict):
    """
    Geeft een overzicht van het aantal uitschieters en percentage uitschieters per kolom.
    """
    overzicht = []
    totaal_rijen = len(df)

    for kolom, (ondergrens, bovengrens) in grenzen.items():
        voorwaarde = (df[kolom] < ondergrens) | (df[kolom] > bovengrens)
        aantal = voorwaarde.sum()
        overzicht.append({
            'kolom': kolom,
            'aantal_uitschieters': aantal,
            'percentage': round(100 * aantal / totaal_rijen, 2)
        })
    return pd.DataFrame(overzicht).sort_values('percentage', ascending = False)

def winsoriseren(df: pd.DataFrame, grenzen: dict):
    """
    Winsorisatie functie: waarden buiten de grenzen worden vervangen 
    door de dichtstbijzijnde grenswaarde.
    """
    df_gecorrigeerd = df.copy()
    for kolom, (ondergrens, bovengrens) in grenzen.items():
        df_gecorrigeerd[kolom] = df_gecorrigeerd[kolom].astype(float)
        df_gecorrigeerd.loc[df_gecorrigeerd[kolom] < ondergrens, kolom] = ondergrens
        df_gecorrigeerd.loc[df_gecorrigeerd[kolom] > bovengrens, kolom] = bovengrens
    return df_gecorrigeerd

In [ ]:
# Song_popularity eerst splitsen van de rest 
x = df.drop(columns=[target])
y = df[target]

In [ ]:
# Kolommen kiezen voor winsorisatie 
kolommen_winsoriseren = ['song_duration_ms', 'loudness', 'tempo', 'time_signature']
#  Grenzen berekenen 
grenzen_voor = ikr_grenzen_berekenen(x[kolommen_winsoriseren])

In [ ]:
# Uitschieters detecteren 
uitschieters_voor = uitschieters_detecteren(x, grenzen_voor)
display(uitschieters_voor)

,kolom,aantal_uitschieters,percentage
3,time_signature,931,6.44
0,song_duration_ms,623,4.31
1,loudness,548,3.79
2,tempo,63,0.44


instrumentalness, speechiness, liveness, danceability, energy, acousticness, audio_valence, energy zijn scores van 0 tot 1. 
Hier niet op winsoriseren, 'uitschieters' hier zijn geen echte uitschieters. 

audio_mode is binair. Niet winsoriseren 

key = 0 tot 11 (toonsoorten). Dit is categorisch numeriek. Niet winsoriseren 

time_signature = van 3-7. Niet winsoriseren. Waardes van 0-2 zijn er al uitgehaald 
song_duration_ms. Winsoriseren 
loudness. Winsoriseren
tempo. Winsoriseren


In [ ]:
# Kopie maken van x dataframe (df zonder target)
df_gecorrigeerd = x.copy()
# Winsorisatie toepassen op geselecteerde kolommen 
df_gecorrigeerd[kolommen_winsoriseren] = winsoriseren(x[kolommen_winsoriseren], grenzen_voor)

In [ ]:
grenzen_na = ikr_grenzen_berekenen(df_gecorrigeerd[kolommen_winsoriseren])
uitschieters_na = uitschieters_detecteren(df_gecorrigeerd, grenzen_na)
display(uitschieters_na)

,kolom,aantal_uitschieters,percentage
0,song_duration_ms,0,0.0
1,loudness,0,0.0
2,tempo,0,0.0
3,time_signature,0,0.0


In [ ]:
# target terugzetten 
df_gecorrigeerd.insert(0, target, y)
display(df_gecorrigeerd.head())

,song_popularity,song_duration_ms,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,audio_mode,speechiness,tempo,time_signature,audio_valence
18816,62,175777.00,0.0128,0.2140,0.148,0.058,7,0.1130,-16.071875,1,0.0370,46.591,4.0,0.0277
15291,46,147373.00,0.9190,0.7100,0.114,0.000,4,0.1540,-12.764000,0,0.2690,47.953,4.0,0.4330
4833,52,230000.00,0.8080,0.1460,0.126,0.949,2,0.3270,-16.071875,1,0.0400,51.607,4.0,0.0356
13248,11,272506.00,0.2750,0.5480,0.493,0.000,10,0.0634,-12.743000,1,0.1760,52.181,4.0,0.6340
5953,47,336517.75,0.6630,0.0594,0.216,0.918,8,0.0451,-16.071875,1,0.0493,54.213,4.0,0.0243


In [ ]:
print(df_gecorrigeerd.select_dtypes(include=['int64', 'float64']).columns)

Index(['song_popularity', 'song_duration_ms', 'acousticness', 'danceability',
       'energy', 'instrumentalness', 'key', 'liveness', 'loudness',
       'audio_mode', 'speechiness', 'tempo', 'time_signature',
       'audio_valence'],
      dtype='object')


In [ ]:
# Train en test set maken 
x = df_gecorrigeerd.drop('song_popularity', axis=1)
y = df_gecorrigeerd['song_popularity']

x_train, x_test, y_train, y_test = train_test_split(
   x, y, test_size=0.2, random_state = 42
)

### Phase 4: Modeling 

Supervised learning, omdat je de uitkomst al hebt 
<br>
Supervised learning heeft 2 hoofdtakken: regressie en classificatie 
<br>
RMSE 
<br>
Meervoudige lineare regressie 
<br>
Logistieke lineare regressie is classification 
<br>
Random forests is het begin van dat machine learning slim werd 